# QM9 KRR Benchmark

Code to perform KRR on the QM9 dataset using different Nystrom methods, using 100k randomly selected molecules as training points. The l1 Laplace kernel is used with a bandwidth 5120, and the regularization parameter is 1e-8; both were chosen using cross-validation. This code was used to produce Figure 3 in the manuscript together with `matlab_plotting/make_krr_plots.m`

In [1]:
# %load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import qml, os
from scipy.io import savemat, loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import plotly.express as px
import plotly.colors as colors

from KRR_Nystrom import KRR_Nystrom
import rpcholesky
import leverage_score
import unif_sample
import matplotlib.pyplot as plt
import time
from functools import partial
import pickle
# util for parallelizing run trials
from joblib import Parallel, delayed

# kernel thinning
# utils for kernel ridge regression
from goodpoints.krr.util_estimators import get_estimator

`eigenpro2` is not installed...
Using `torch.linalg.solve` for training the kernel model

          and may cause an `Out-of-Memory` error
`eigenpro2` is a more scalable solver. To use, pass `method="eigenpro"` to `model.fit()`
To install `eigenpro2` visit https://github.com/EigenPro/EigenPro-pytorch/tree/pytorch/


/home/ag2435/.conda/envs/rpcholesky/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import joblib
print(joblib.__version__)
print(joblib.cpu_count())

1.2.0
8


In [4]:
# add this to be able to render plotly plots in non-vscode notebooks
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [5]:

def get_molecules(directory = "molecules/", max_atoms = 29, max_mols = np.Inf, output_index = 7):
    compounds = []
    energies = []
    for f in sorted(os.listdir("molecules/")):
        if len(compounds) >= max_mols:
            break

        try:
            mol = qml.Compound(xyz="molecules/"+f)
            mol.generate_coulomb_matrix(size=max_atoms, sorting="row-norm")
            with open("molecules/"+f) as myfile:
                line = list(myfile.readlines())[1]
                energies.append(float(line.split()[output_index]) * 27.2114) # Hartrees to eV
            compounds.append(mol)
        except ValueError:
            pass
    
    c = list(zip(compounds, energies))
    np.random.shuffle(c)
    compounds, energies = zip(*c)

    X = np.array([mol.representation for mol in compounds])
    Y = np.array(energies).reshape((X.shape[0],1))

    return X, Y 
    

In [6]:
if not os.path.isfile("data/homo.mat"):
    X, Y = get_molecules()
    data = { "X" : X, "Y" : Y }
    savemat("data/homo.mat", data)
else:
    data = loadmat("data/homo.mat")
    
feature = data['X']
target = data['Y'].flatten()
scaler = StandardScaler()
feature = scaler.fit_transform(feature)
n,d = np.shape(feature)


In [7]:
# From KT compress
def log4(n):
    return np.log2(n) / 2
def get_g(n):
    return int( np.ceil( log4(log4(n)) ) ) # Use default value
def largest_power_of_four(n):
    """Returns largest power of four less than or equal to n
    """
    return 4**( (n.bit_length() - 1 )//2)

def get_coreset_size(n, m=1):
    if get_g(n) <= m:
        # with TicToc('compresspp', print_toc=PRINT_TOC):
        # Compress with g'=g+inflation (compressing returns set of size 2^(g+inflation) sqrt(n) )
        # Thin with g'=g (thinning returns set of size 2^inflation sqrt(n) )
        largest_pow_four = largest_power_of_four(n)
        log2n = n.bit_length() - 1
        scale = n // largest_pow_four
        return 2**( 2*(log2n//2) - m ) * scale
    else:
        return int(n / 2**m)

In [8]:

num_train = 20000 # 100000
num_test = 20000 # n - num_train
# ks = range(200, 1200, 200)
sqrt_n = get_coreset_size(num_train, int(log4(num_train)))
ks = [ sqrt_n, sqrt_n* 2, sqrt_n*4 ]
print('ks', ks)

train_sample = feature[:num_train]
train_sample_target = target[:num_train]
test_sample = feature[num_train:num_train+num_test]
test_sample_target = target[num_train:num_train+num_test]


ks [128, 256, 512]


In [9]:

def mean_squared_error(true, pred):
    return np.mean((true - pred)**2)
def mean_average_error(true, pred):
    return np.mean(np.abs(true - pred))
def SMAPE(true,pred):
    return np.mean(abs(true - pred)/((abs(true)+abs(pred))/2))


In [10]:

methods = { 
    # deterministic methods
    'Greedy' : rpcholesky.greedy, 
    
    # random methods
    'Uniform' : unif_sample.uniform_sample,
    'RPCholesky' : rpcholesky.rpcholesky,
    'RLS' : leverage_score.recursive_rls_acc,
    'block50RPCholesky' : partial(rpcholesky.block_rpcholesky,b=50),

    'kt': None,
    'st' : None
}

num_trials = 100
lamb = 1.0e-8
sigma = 5120.0
result = dict()
# n_jobs = 2 
n_jobs = 8
savepath = f"data/molecule{num_train // 1000}k-trials={num_trials}.pkl"

In [11]:
def train_predict(name, method, train_sample, test_sample, k, idx_k):
    start = time.time()
    if name == 'Greedy':
        model = KRR_Nystrom(kernel = "laplace", 
                    bandwidth = sigma)
        model.fit_Nystrom(train_sample, train_sample_target, lamb = lamb, sample_num = k, sample_method = method, solve_method = solve_method)
        preds = model.predict_Nystrom(test_sample)
    elif name in ['kt', 'st']: # our methods
        while True:
            try:
                # print(f"Trial {i}")
                model = get_estimator(
                    'regression', 
                    name.lower(), 
                    kernel='laplace',
                    alpha=lamb,
                    sigma=sigma,
                    m=int(log4(num_train))-idx_k,
                )

                model.fit(train_sample, train_sample_target)
                assert len(model.sol_) == k, f"len(model.sol_)={len(model.sol_)} should be same as k={k}"
                preds = model.predict(test_sample)

                break
            except np.linalg.LinAlgError:
                continue
    else:
        while True:
            try:
                # print(f"Trial {i}")
                # # Original
                # model = KRR_Nystrom(kernel = "gaussian", bandwidth = sigma)
                # # Bug fix
                model = KRR_Nystrom(kernel = "laplace", bandwidth = sigma)
                
                model.fit_Nystrom(train_sample, train_sample_target, lamb = lamb, sample_num = k, sample_method = method, solve_method = solve_method)
                preds = model.predict_Nystrom(test_sample)
                break
            except np.linalg.LinAlgError:
                continue
    end = time.time()
    return preds, end - start
    # return preds

In [ ]:

solve_method = 'Direct'

for name, method in methods.items():
    result[name] = dict()
    print(f'------------- Method: {name} -------------')
    result[name]["trace_errors"] = {} #np.zeros((len(ks),2))
    result[name]["KRRMSE"] = {} #np.zeros((len(ks),2))
    result[name]["KRRMAE"] = {} #np.zeros((len(ks),2))
    result[name]["KRRSMAPE"] = {}   #np.zeros((len(ks),2))
    result[name]["queries"] = {}    #np.zeros((len(ks),2))

    for idx_k in range(len(ks)):
        k = ks[idx_k]
        print(f'k = {k}')
        trace_err = []
        runtime = []
        queries = []
        KRRmse = []
        KRRmae = []
        KRRsmape = []

        if name == 'Greedy':
            trials = 1 # deterministic
        else:
            trials = num_trials # stochastic

        parallel = Parallel(n_jobs= n_jobs) #, return_as="generator") need joblib>=1.3 for return_as functionality
        output_generator = parallel(delayed(train_predict)(
            name, 
            method, 
            train_sample, 
            test_sample,
            k,
            idx_k,
        ) for _ in range(trials))

        for preds, elapsed_time in output_generator:
            KRRmse.append(mean_squared_error(test_sample_target, preds))
            KRRmae.append(mean_average_error(test_sample_target, preds))
            KRRsmape.append(SMAPE(test_sample_target, preds))
            # queries.append(model.queries)
            # trace_err.append(model.reltrace_err) 
            
            # TODO: placeholder for now
            queries.append(np.nan)
            trace_err.append(np.nan)

            print(f'KRR acc: mse {KRRmse[-1]}, mae {KRRmae[-1]}, smape {KRRsmape[-1]}')
            # print(f'time: sample {model.sample_time}, linsolve {model.linsolve_time}, pred {model.pred_time}')
            print(f'time: {elapsed_time}')
            
        result[name]["trace_errors"][k] = trace_err   # [np.mean(trace_err),np.std(trace_err)]
        result[name]["KRRMSE"][k] = KRRmse    # [np.mean(KRRmse),np.std(KRRmse)]
        result[name]["KRRMAE"][k] = KRRmae    #[np.mean(KRRmae),np.std(KRRmae)]
        result[name]["KRRSMAPE"][k] = KRRsmape    #[np.mean(KRRsmape),np.std(KRRsmape)]
        result[name]["queries"][k] = queries  #[np.mean(queries)/float(num_train**2),np.std(queries)/float(num_train**2)]

        # savemat("data/{}_molecule100k.mat".format(name), result[name])
        # use pickle to save the result periodically
        with open(savepath, 'wb') as f:
            pickle.dump(result, f)


In [ ]:
result

## Plot results

In [ ]:
with open(savepath, "rb") as f:
    result = pickle.load(f)
# with open("data/molecule20k-kt.pkl", "rb") as f:
#     result2 = pickle.load(f)
# concatenate the results
# result.update(result2)

In [ ]:
# plot line graph with error bars
metrics = ['KRRSMAPE', 'KRRMSE']
fig = make_subplots(rows=len(metrics), cols=1, subplot_titles=metrics, shared_xaxes=True, vertical_spacing=0.1)
colors_list = colors.qualitative.Plotly 
# * (
#     len(model_names) // len(colors.qualitative.Plotly) + 1
# )
model_names = list(result.keys())

for name in result.keys():
    color = colors_list[model_names.index(name)]
    for r, metric in enumerate(metrics):
        if name in ['kt', 'st']:
            for k, vals in result[name][metric].items():
                # print(k, krrmse)
                fig.add_trace(go.Box(
                    x=[k] *len(vals),
                    y=vals,
                    name=name,
                    # opacity=0.5,
                    legendgroup=name,
                    # line_color=color,
                    # offsetgroup=model_name_prefix,
                    # showlegend=color not in colors_used,
                    boxmean=True,
                    line_color = color
                ), row=r+1, col=1)
        else:
            means = [np.mean(krrmse) for _, krrmse in result[name][metric].items() ]
            stds = [np.std(krrmse) for _, krrmse in result[name][metric].items() ]
            # print(ks, means, stds)
            fig.add_trace(go.Scatter(
                x=ks, 
                y=means, 
                mode='lines+markers', 
                name=name, 
                error_y=dict(
                    type='data', # value of error bar given in data coordinates
                    array=stds,
                    visible=True
                ),
                legendgroup=name,
                line_color = color
            ), row=r+1, col=1)
    fig.update_xaxes(title_text="k", row=r+1, col=1)
        
fig.update_layout(title='MSE vs. k (columns)',
                    #  xaxis_title='Coreset Size',
                    #  yaxis_title='MSE',
                     height=800, width=800)
fig.show()